## 기본 베이지 DNN

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
import os
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support as sk
from sklearn.metrics import f1_score ## F1 Score 구하기
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
   
%matplotlib inline

C:\Users\LeeInGyu\anaconda3\envs\dnn\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class DNNModel(nn.Module):
    def __init__(self):
        super(DNNModel, self).__init__()
        self.input_layer = nn.Linear(6, 128)
        self.hidden_layer1 = nn.Linear(128, 256)
        self.hidden_layer2 = nn.Linear(256, 128)
        self.output_layer   = nn.Linear(128,3)
        self.relu = nn.ReLU()

    def forward(self, x):
        out =  self.relu(self.input_layer(x))
        out =  self.relu(self.hidden_layer1(out))
        out =  self.relu(self.hidden_layer2(out))
        out =  self.output_layer(out)
        return out 

# device 설정 (cuda:0 혹은 cpu)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = DNNModel() # Model 생성
model.to(device)   # device 에 로드 (cpu or cuda)

# 옵티마이저를 정의합니다. 옵티마이저에는 model.parameters()를 지정해야 합니다.
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 손실함수(loss function)을 지정합니다. Multi-Class Classification 이기 때문에 CrossEntropy 손실을 지정하였습니다.
loss_fn = nn.CrossEntropyLoss()

In [3]:
from tqdm import tqdm  # Progress Bar 출력

def model_train(model, data_loader, loss_fn, optimizer, device):
    # 모델을 훈련모드로 설정합니다. training mode 일 때 Gradient 가 업데이트 됩니다. 반드시 train()으로 모드 변경을 해야 합니다.
    model.train()
    # loss와 accuracy 계산을 위한 임시 변수 입니다. 0으로 초기화합니다.
    running_loss = 0
    corr = 0

    # 예쁘게 Progress Bar를 출력하면서 훈련 상태를 모니터링 하기 위하여 tqdm으로 래핑합니다.
    prograss_bar = tqdm(data_loader)

    # mini-batch 학습을 시작합니다.
    for data, lbl in prograss_bar:
#     for data, lbl in data_loader:
        # image, label 데이터를 device에 올립니다.
        data, lbl = data.to(device), lbl.to(device)
        # 누적 Gradient를 초기화 합니다.
        optimizer.zero_grad()

        # Forward Propagation을 진행하여 결과를 얻습니다.
        output = model(data)

        # 손실함수에 output, label 값을 대입하여 손실을 계산합니다.
        loss = loss_fn(output, lbl)
        # 오차역전파(Back Propagation)을 진행하여 미분 값을 계산합니다.
        loss.backward()

        # 계산된 Gradient를 업데이트 합니다.
        optimizer.step()

        # output의 max(dim=1)은 max probability와 max index를 반환합니다.
        # max probability는 무시하고, max index는 pred에 저장하여 label 값과 대조하여 정확도를 도출합니다.
        _, pred = output.max(dim=1)
        # pred.eq(lbl).sum() 은 정확히 맞춘 label의 합계를 계산합니다. item()은 tensor에서 값을 추출합니다.
        # 합계는 corr 변수에 누적합니다.
        corr += pred.eq(lbl).sum().item()
        
        # loss 값은 1개 배치의 평균 손실(loss) 입니다. data.size(0)은 배치사이즈(batch size) 입니다.
        # loss 와 data.size(0)를 곱하면 1개 배치의 전체 loss가 계산됩니다.
        # 이를 누적한 뒤 Epoch 종료시 전체 데이터셋의 개수로 나누어 평균 loss를 산출합니다.
        running_loss += loss.item() * data.size(0)

    # 누적된 정답수를 전체 개수로 나누어 주면 정확도가 산출됩니다.
    acc = corr / len(data_loader.dataset)
    # 평균 손실(loss)과 정확도를 반환합니다.
    # train_loss, train_acc
    return running_loss / len(data_loader.dataset), acc

In [4]:
def model_evaluate(model, data_loader, loss_fn, device):
    # model.eval()은 모델을 평가모드로 설정을 바꾸어 줍니다. 
    # dropout과 같은 layer의 역할 변경을 위하여 evaluation 진행시 꼭 필요한 절차 입니다.
    model.eval()
    # Gradient가 업데이트 되는 것을 방지 하기 위하여 반드시 필요합니다.
    with torch.no_grad():
        # loss와 accuracy 계산을 위한 임시 변수 입니다. 0으로 초기화합니다.
        corr = 0
        running_loss = 0

        # 배치별 evaluation을 진행합니다.
        for data, lbl in data_loader:
            # device에 데이터를 올립니다.
            data, lbl = data.to(device), lbl.to(device)

            # 모델에 Forward Propagation을 하여 결과를 도출합니다.
            output = model(data)

            # output의 max(dim=1)은 max probability와 max index를 반환합니다.
            # max probability는 무시하고, max index는 pred에 저장하여 label 값과 대조하여 정확도를 도출합니다.
            _, pred = output.max(dim=1)
            

            # pred.eq(lbl).sum() 은 정확히 맞춘 label의 합계를 계산합니다. item()은 tensor에서 값을 추출합니다.
            # 합계는 corr 변수에 누적합니다.
            corr += torch.sum(pred.eq(lbl)).item()
            
            # loss 값은 1개 배치의 평균 손실(loss) 입니다. data.size(0)은 배치사이즈(batch size) 입니다.
            # loss 와 data.size(0)를 곱하면 1개 배치의 전체 loss가 계산됩니다.
            # 이를 누적한 뒤 Epoch 종료시 전체 데이터셋의 개수로 나누어 평균 loss를 산출합니다.
            running_loss += loss_fn(output, lbl).item() * data.size(0)

        # validation 정확도를 계산합니다.
        # 누적한 정답숫자를 전체 데이터셋의 숫자로 나누어 최종 accuracy를 산출합니다.
        acc = corr / len(data_loader.dataset)

        # 결과를 반환합니다.
        # val_loss, val_acc
        return running_loss / len(data_loader.dataset), acc

In [5]:
def model_test(model, data_loader, loss_fn, device):
    # model.eval()은 모델을 평가모드로 설정을 바꾸어 줍니다. 
    # dropout과 같은 layer의 역할 변경을 위하여 evaluation 진행시 꼭 필요한 절차 입니다.
    model.eval()
    pred_list=[]
    # Gradient가 업데이트 되는 것을 방지 하기 위하여 반드시 필요합니다.
    with torch.no_grad():
        # loss와 accuracy 계산을 위한 임시 변수 입니다. 0으로 초기화합니다.
        corr = 0
        running_loss = 0

        # 배치별 evaluation을 진행합니다.
        for data, lbl in data_loader:
            # device에 데이터를 올립니다.
            data, lbl = data.to(device), lbl.to(device)

            # 모델에 Forward Propagation을 하여 결과를 도출합니다.
            output = model(data)
            
            
            # output의 max(dim=1)은 max probability와 max index를 반환합니다.
            # max probability는 무시하고, max index는 pred에 저장하여 label 값과 대조하여 정확도를 도출합니다.
            _, pred = output.max(dim=1)
            pred_array = pred.tolist()
            pred_list.append(pred_array) # confusion matrix를 위해 pred 리턴 값
            # pred.eq(lbl).sum() 은 정확히 맞춘 label의 합계를 계산합니다. item()은 tensor에서 값을 추출합니다.
            # 합계는 corr 변수에 누적합니다.
            corr += torch.sum(pred.eq(lbl)).item()
            
            # loss 값은 1개 배치의 평균 손실(loss) 입니다. data.size(0)은 배치사이즈(batch size) 입니다.
            # loss 와 data.size(0)를 곱하면 1개 배치의 전체 loss가 계산됩니다.
            # 이를 누적한 뒤 Epoch 종료시 전체 데이터셋의 개수로 나누어 평균 loss를 산출합니다.
            running_loss += loss_fn(output, lbl).item() * data.size(0)

        # validation 정확도를 계산합니다.
        # 누적한 정답숫자를 전체 데이터셋의 숫자로 나누어 최종 accuracy를 산출합니다.
        acc = corr / len(data_loader.dataset)

        # 결과를 반환합니다.
        # val_loss, val_acc
        return running_loss / len(data_loader.dataset), acc, pred_list

In [6]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [ ]:
for file in os.listdir('./1term_data'):
    
    filename=file.rstrip('.xlsx')
    #Dataframe 읽고 X y 나누기
    df=pd.read_excel('./1term_data/'+file)
    X=df.iloc[:,[1,3,4,5,6,7]]
    y=df.iloc[:,-1]
    
    #Scaling
    scaler = StandardScaler()
    X_scaler = scaler.fit_transform(X)
    
    #SMOTE 적용
    smote = SMOTE(random_state=0)
    X_train_over,y_train_over = smote.fit_resample(X_scaler,y)
#     print('SMOTE 적용 전 레이블 값 분포: \n', pd.Series(y).value_counts())
#     print('SMOTE 적용 후 레이블 값 분포: \n', pd.Series(y_train_over).value_counts())
    
    #kFold
    skf=StratifiedKFold(n_splits=10,shuffle=True)
    
    num_epochs = 300
    i=0    
    
    #결과 넣을 배열
    most_acc=[]
    empty=pd.DataFrame()
    CMResult=[[[0 for k in range(3)]for j in range(3)] for i in range(10)]
    Result=[[0 for j in range(4)] for i in range(10)]
    loss_list=[[0 for j in range(num_epochs)] for i in range(10)]
    acc_list=[[0 for j in range(num_epochs)] for i in range(10)]
    AverageCM=[[0 for i in range(3)] for j in range(3)]
    torch.save(model.state_dict(), 'DNNModel.pth')
    for train_index, test_index in skf.split(X_train_over,y_train_over):
        model.load_state_dict(torch.load('DNNModel.pth'))
        X_train, X_test = X_train_over[train_index], X_train_over[test_index]
        y_train, y_test = y_train_over[train_index], y_train_over[test_index]

        X_train = torch.FloatTensor(X_train)
        X_test = torch.FloatTensor(X_test)
        y_train = torch.LongTensor(y_train.to_numpy())
        y_test = torch.LongTensor(y_test.to_numpy())

        train_dataset = TensorDataset(X_train, y_train)
        test_dataset=TensorDataset(X_test, y_test)

        train_dataloader = DataLoader(train_dataset, batch_size=500,shuffle=False)
        test_dataloader = DataLoader(test_dataset, batch_size=200,shuffle=False)

        # 최대 Epoch을 지정합니다.
        min_loss = 9999999999999
        # Epoch 별 훈련 및 검증을 수행합니다.
        temp_loss=[]
        temp_acc=[]
        for epoch in range(num_epochs):
            
            # Model Training
            # 훈련 손실과 정확도를 반환 받습니다.
            train_loss, train_acc = model_train(model, train_dataloader, loss_fn, optimizer, device)

            # 검증 손실과 검증 정확도를 반환 받습니다.
            val_loss, val_acc = model_evaluate(model, test_dataloader, loss_fn, device)   

            # val_loss 가 개선되었다면 min_loss를 갱신하고 model의 가중치(weights)를 저장합니다.
#             if val_loss < min_loss:
# #                     print(f'[INFO] val_acc has been improved from {min_loss:.5f} to {val_loss:.5f}. Saving Model!')
#                     min_loss = val_loss
            
            # Epoch 별 결과를 출력합니다.
            if(epoch%50==0):
                print(f'epoch {epoch+1:02d}, loss: {train_loss:.5f}, acc: {train_acc:.5f}, val_loss: {val_loss:.5f}, val_accuracy: {val_acc:.5f}')
            temp_loss.append(train_loss)
            temp_acc.append(train_acc)
            
        ## 저장한 가중치 로드 후 검증 성능 측정
        
        loss_list[i]=temp_loss
        acc_list[i]=temp_acc
              

        # 최종 검증 손실(validation loss)와 검증 정확도(validation accuracy)를 산출합니다.
#         print("{}번째 교차검증".format(i))
        final_loss, final_acc, pred_list = model_test(model, test_dataloader, loss_fn, device)
#         print(f'evaluation loss: {final_loss:.5f}, evaluation accuracy: {final_acc:.5f}')
        
        pred_list=flatten_list(pred_list)
        pred_list=np.array(pred_list)
        pred_list=pred_list.astype(np.int64)
        y_test=y_test.numpy()
        y_test_list=y_test.tolist()
#       --------------------------------------------------------------------------------
        #여기까지 pred_list : prediction, y_test : 실제값                          
        
        df_pred=(pd.DataFrame(pred_list)).value_counts()
        df_pred.columns=['Prediction']
        df_real=(pd.DataFrame(y_test)).value_counts()
        df_real.columns=['Real']
        df_values=pd.concat([df_pred,df_real],axis=1)
        
        path='./Result/ValueCount/'+filename
        isExist = os.path.exists(path)
        if not isExist:
            os.makedirs(path)     
        df_values.to_csv('./Result/ValueCount/'+filename+'/'+filename+str(i)+'value_count.csv')
        
        accuracy=accuracy_score(y_test, pred_list)
#         print(filename + "[{}]Accuracy : {}".format(i,accuracy))   
        
        #f1score
        f1 = f1_score(y_test,pred_list, average='weighted')
#         print(filename + "[{}]F1score : {}".format(i,f1))
        
        #precision/recall
        p_rlist=sk(y_test,pred_list,average='weighted')
#         print(filename + "[{}]Precision : {}".format(i,p_rlist[0]))
#         print(filename + "[{}]Recall : {}".format(i,p_rlist[1]))
        
                                  
        #결과 배열에 넣기
        Result[i][0]=accuracy
        Result[i][1]=f1
        Result[i][2]=p_rlist[0]
        Result[i][3]=p_rlist[1]
        
        cm=confusion_matrix(y_test, pred_list,normalize="true")
        CMResult[i]=cm
        
        disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=[0,1,2])
        disp.plot()
        path='./ConfusionMatrix/'+filename
        isExist = os.path.exists(path)
        if not isExist:
            os.makedirs(path)     
        plt.savefig("./ConfusionMatrix/"+filename+"/"+filename+"_"+str(i)+"_CM.png")
#         plt.show()
        i=i+1
        
    df_result=pd.DataFrame(Result,columns=['Accuracy','F1score','Precision','Recall'])
    df_result_mean=df_result.mean()
    df_result2=pd.concat((df_result,df_result_mean),axis=1,ignore_index=True,)
    df_result2.to_csv('./Result/'+filename+'_Result.csv')    
    
    
    AverageCM=[[0 for i in range(3)] for j in range(3)]
    for i in range(10):
        for j in range(3):
            for k in range(3):
                AverageCM[j][k]=AverageCM[j][k]+CMResult[i][j][k]
    
    for j in range(3):
        for k in range(3):
            AverageCM[j][k]=AverageCM[j][k]/10
    
    AverageCM=np.array(AverageCM)           
    disp = ConfusionMatrixDisplay(confusion_matrix=AverageCM,display_labels=[0,1,2])
    disp.plot()
    plt.savefig("./ConfusionMatrix/AverageConfusionMatrix/"+filename+".png")
#     plt.show()
    
    df_loss=pd.DataFrame(loss_list)
    df_loss.to_csv('./Loss/'+filename+'_loss.csv')
    
    df_acc=pd.DataFrame(acc_list)
    df_acc.to_csv('./Accuracy/'+filename+'_accuracy.csv')
    
    del Result
    del CMResult
    del loss_list
    del acc_list
    del AverageCM

100%|██████████████████████████████████████████████████████████████████████████████████| 87/87 [00:01<00:00, 80.24it/s]


epoch 01, loss: 0.99980, acc: 0.58696, val_loss: 1.90124, val_accuracy: 0.33333


100%|█████████████████████████████████████████████████████████████████████████████████| 87/87 [00:00<00:00, 173.61it/s]


epoch 51, loss: 0.70187, acc: 0.68125, val_loss: 0.73422, val_accuracy: 0.69756


100%|█████████████████████████████████████████████████████████████████████████████████| 87/87 [00:00<00:00, 206.60it/s]


...

100%|█████████████████████████████████████████████████████████████████████████████████| 87/87 [00:00<00:00, 202.28it/s]


epoch 01, loss: 1.02762, acc: 0.62374, val_loss: 5.23934, val_accuracy: 0.33333


100%|█████████████████████████████████████████████████████████████████████████████████| 87/87 [00:00<00:00, 207.10it/s]


epoch 51, loss: 0.75183, acc: 0.67502, val_loss: 0.74133, val_accuracy: 0.67335


 47%|██████████████████████████████████████▏                                          | 41/87 [00:00<00:00, 207.60it/s]